# **S2_Patient Alerts by Payor Report**

# **Part 1 Process Data**

### **Import Raw File**

In [50]:
import pandas as pd
import numpy as np
import os
import sys
from datetime import datetime
import shutil

In [51]:
#input aco and dateframe
deployment=input('Enter the name of ACO ')
date_input=input('Enter the date frame as mmdd-mmdd: ')
path=fr"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\{date_input} {deployment}"
if deployment=='PBACO':
    filename=f'Alerts_by_Payor'
else:
    filename=f'{deployment}_Alerts_by_Payor' 

Enter the name of ACO  Empire
Enter the date frame as mmdd-mmdd:  1223-1229


In [52]:
#import raw report,delete header
if deployment=='PBACO':
    df=pd.read_excel(fr"{path}\Patient Alerts by NPI and Payor.xlsx",skiprows=8)
else:
    df=pd.read_excel(fr"{path}\{deployment} Patient Alerts by NPI and Payor.xlsx",skiprows=8)
Unnamed_col=[col for col in df.columns if 'Unnamed' in col]
df=df.drop(Unnamed_col,axis=1)

### **Convert DataType**

In [53]:
df['TIN']=df['TIN'].astype('Int64')
df['NPI']=df['NPI'].astype('Int64')

#change percentage
columns= ['Pts With Actions  Under 3BD(%)','Pts With Actions(%)','Pts Read  Under 3BD(%)','Pts Read(%)']
for column in columns:
    df[column] = (df[column].astype(float) * 100).map(lambda x: '{:.2f}%'.format(x))

### **Delete Duplicate NPI with Consultant Name as 'Duplicates', 'no Consultants', and 'None'**

In [54]:
drop_rows=df[(df.duplicated(subset=['NPI','Payor'],keep=False)) & (df['Provider Consultant'].isin(['Duplicate','no consultant','None']))].index
df.drop(index=drop_rows,inplace=True)

### **Look Up Blank Things, Get TIN,TIN Name for Blank TINs, Get Consultant Name for All Consultants**

In [55]:
#import blank thing
BlankThings=pd.read_excel(r"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\Derrell Patient Alerts Macro.xlsm",sheet_name=3)

In [56]:
#get TIN,TIN Name from blankthings if TIN is blank, get consultant from blankthings
df2=df[df['Provider Consultant'].isin(['Duplicate','no consultant','None'])].merge(BlankThings.drop_duplicates('NPI'), on='NPI',how='left',suffixes=('', '_right'))
df2['TIN']=df2['TIN_right'].combine_first(df2['TIN'])
df2['TIN Name']=df2['TIN Name_right'].combine_first(df2['TIN Name'])
df2['Provider Consultant']=df2['Provider Consultant_right']

In [57]:
#remove duplicate columns from blankthings in df2
column_order = df.columns.tolist()
df2 = df2[column_order + [col for col in df2.columns if col not in column_order]]
df2=df2.iloc[:,:-5]

In [58]:
#drop 'Duplicate','no consultant','None' rows in df then union df2
drop_index=df[df['Provider Consultant'].isin(['Duplicate','no consultant','None'])].index
df.drop(index=drop_index,inplace=True)
df=pd.concat([df,df2],ignore_index=True)

### **Delete Duplicate Rows**

In [59]:
#remove 'delete' rows
df=df[df['Provider Consultant']!='DELETE']

In [60]:
#remove duplicates if PC is David T or Emily
drop_rows=df[(df.duplicated(subset=['NPI','Payor'],keep=False)) & (df['Provider Consultant'].isin(['David T']))].index
df.drop(index=drop_rows,inplace=True)
drop_rows=df[(df.duplicated(subset=['NPI','Payor'],keep=False)) & (df['Provider Consultant'].isin(['Emily']))].index
df.drop(index=drop_rows,inplace=True)

In [61]:
#sort data
df=df.sort_values(by=['Provider Consultant','TIN Name','Provider','Pts Sent To App'],ascending=[True,True,True,False])

In [62]:
#remove other duplicates NPI
drop_rows=df[df.duplicated(subset=['Payor','NPI'],keep='first')].index
df.drop(index=drop_rows,inplace=True)

In [63]:
#export
df.to_excel(fr'{path}\{filename}_{date_input}.xlsx', index=False, sheet_name='Sheet1')
print(f'Step 1 Finished, file saved at: {path}\\{filename}_{date_input}.xlsx, look up N/A consultant in Act! before Next Step')
continue_input=input('Enter Y when you are ready to continue')
if continue_input.lower()=='y':
    print('Start adding header back')
else:
    sys.exit()

Step 1 Finished, file saved at: C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\1223-1229 Empire\Empire_Alerts_by_Payor_1223-1229.xlsx, look up N/A consultant in Act! before Next Step


Enter Y when you are ready to continue Y


Start adding header back


# **Part 2 Add The Header Back**

In [64]:
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import Alignment
from openpyxl.styles import Font
from openpyxl.styles import Border, Side
from openpyxl.styles import PatternFill
from openpyxl.utils import range_boundaries
from openpyxl.styles import NamedStyle

### **Import Report Header and Body**

In [65]:
header_wb=load_workbook(r"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\Derrell Patient Alerts Macro.xlsm")
body_wb=load_workbook(fr'{path}\{filename}_{date_input}.xlsx')
# Select the active sheet in both workbooks
header_ws = header_wb['doc-header'] 
body_ws = body_wb.active

### **Format Body**

In [66]:
#fill colors and set size
pt_sent_to_app_fill = PatternFill(start_color='6495ED', end_color='6495ED', fill_type='solid')
for row in range(1, body_ws.max_row + 1):  # Use max_row to iterate through actual data rows
    body_ws.cell(row=row, column=8).fill = pt_sent_to_app_fill
    
pt_with_actions_fill = PatternFill(start_color='90EE90', end_color='90EE90', fill_type='solid')
for col in range(9, 16):  # columns I to O
    for row in range(1, body_ws.max_row + 1):
        body_ws.cell(row=row, column=col).fill = pt_with_actions_fill
        
pt_with_reads_fill = PatternFill(start_color='F0E68C', end_color='F0E68C', fill_type='solid')
for col in range(16, 23):  # columns P to V
    for row in range(1, body_ws.max_row + 1):
        body_ws.cell(row=row, column=col).fill = pt_with_reads_fill

bsa_fill = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')
for col in range(23, 27):  # columns W to Z
    for row in range(1, body_ws.max_row + 1):
        body_ws.cell(row=row, column=col).fill = bsa_fill

login_fill = PatternFill(start_color='AFEEEE', end_color='AFEEEE', fill_type='solid')
for row in range(1, body_ws.max_row + 1):
    body_ws.cell(row=row, column=27).fill = login_fill
    
for row in range(1, body_ws.max_row + 1): #setup body rows
    for cell in body_ws[row]:  
        cell.font = Font(name='Segoe UI', size=8) 
        cell.alignment = Alignment(horizontal='left')
        
for cell in body_ws[1]:  #set up body header
    cell.alignment = Alignment(wrap_text=True, vertical='bottom', horizontal='left', text_rotation=90)
    cell.font = Font(name='Segoe UI', size=8, bold=True)

gray_color = 'D3D3D3'  # set up body boarder
thin_gray_border = Border(
    left=Side(style='thin', color=gray_color),
    right=Side(style='thin', color=gray_color),
    top=Side(style='thin', color=gray_color),
    bottom=Side(style='thin', color=gray_color)
)
for row in body_ws.iter_rows(min_row=1, max_row=body_ws.max_row):
    for cell in row:
        cell.border = thin_gray_border   

### **Insert Header to Body**

In [67]:
#insert header
body_ws.insert_rows(1, 5)

# Copy values, fonts, styles, and merged cells from the header to the body
for header_row in range(1, 6):  # Loop through the top 5 rows
    for col in range(1, header_ws.max_column + 1):
        header_cell = header_ws.cell(row=header_row, column=col)
        body_cell = body_ws.cell(row=header_row, column=col)
        
        # Copy the value
        body_cell.value = header_cell.value
        
        # Copy individual styles if the cell has them
        body_cell.font = Font(
            name=header_cell.font.name,
            size=header_cell.font.size,
            bold=header_cell.font.bold,
            italic=header_cell.font.italic,
            vertAlign=header_cell.font.vertAlign,
            underline=header_cell.font.underline,
            strike=header_cell.font.strike,
            color=header_cell.font.color
        )
        body_cell.alignment = Alignment(
            horizontal=header_cell.alignment.horizontal,
            vertical=header_cell.alignment.vertical,
            wrap_text=header_cell.alignment.wrap_text
        )
        body_cell.border = Border(
            left=header_cell.border.left,
            right=header_cell.border.right,
            top=header_cell.border.top,
            bottom=header_cell.border.bottom
        )
        body_cell.fill = PatternFill(
            fill_type=header_cell.fill.fill_type,
            start_color=header_cell.fill.start_color,
            end_color=header_cell.fill.end_color
        )

### **Merge header cells**

In [68]:
# Function to extract range boundaries from merged cells
def range_boundaries(merged_range):
    """Extracts the boundaries of a merged cell range."""
    min_col, min_row, max_col, max_row = openpyxl.utils.range_boundaries(merged_range)
    max_col=body_ws.max_column
    return min_col, min_row, max_col, max_row

# Copy merged cells from header to body
for merged_range in header_ws.merged_cells.ranges:
    min_col, min_row, max_col, max_row = range_boundaries(str(merged_range))
    if min_row <= 5:  # Only copy merged cells in the first 5 rows
        body_ws.merge_cells(start_row=min_row,
                             start_column=min_col,
                             end_row=max_row,
                             end_column=max_col)

### **Replace header value**

In [69]:
if 'ssss' in body_ws['A2'].value:  
    body_ws['A2'].value = body_ws['A2'].value.replace('ssss', fr"{date_input[:2]}/{date_input[2:4]}/{datetime.now().year}")
if 'ffff' in body_ws['A2'].value:  
    body_ws['A2'].value = body_ws['A2'].value.replace('ffff', fr"{date_input[5:7]}/{date_input[7:9]}/{datetime.now().year}")   
if 'dddd' in body_ws['A3'].value:  
    body_ws['A3'].value = body_ws['A3'].value.replace('dddd', datetime.now().date().strftime('%Y-%m-%d'))
if 'xxxx' in body_ws['A4'].value:  
    body_ws['A4'].value = body_ws['A4'].value.replace('xxxx', f"{round((df[df['Provider Consultant']!='David T']['Total Pts With Actions'].sum()/df[df['Provider Consultant']!='David T']['Pts Sent To App'].sum())*100,2)}") 
if 'yyyy' in body_ws['A4'].value:  
    body_ws['A4'].value = body_ws['A4'].value.replace('yyyy', f"{round((df[df['Provider Consultant']!='David T']['Total Pts Read'].sum()/df[df['Provider Consultant']!='David T']['Pts Sent To App'].sum())*100,2)}") 

### **Adjust the size for each cell**

In [70]:
#format size for each cell
body_ws.row_dimensions[1].height = 30.8
body_ws.row_dimensions[5].height = 200
body_ws.row_dimensions[6].height = 86

column_widths = [10,9, 32, 9, 22, 9, 4, 4, 6.22, 6, 4, 8.22, 8.22, 6.33, 6, 6.22, 4, 4, 6.22, 6.22, 6.22, 4, 6, 4, 6, 5, 4]
columns = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ') + ['AA']
for i, width in enumerate(column_widths):
    body_ws.column_dimensions[columns[i]].width = width

### **Hide grid, add filter, and change title name**

In [71]:
#hide grid in header
fill = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")
for row in range(1, 6):
    for col in range(1, body_ws.max_column):
        body_ws.cell(row=row, column=col).fill = fill


#invisiable boarder in row 6
invisible_border = Border(left=Side(style='none'),
                          right=Side(style='none'),
                          top=Side(style='none'),
                          bottom=Side(style='none'))

for col in range(1, body_ws.max_column+1): 
    body_ws.cell(row=6, column=col).border = invisible_border
    
body_ws.auto_filter.ref = "A6:AA6"
body_ws.title='All'

In [72]:
#export
body_wb.save(fr'{path}\{filename}_{date_input}.xlsx')

os.makedirs(fr'{path}\Raw_Files', exist_ok=True)
if deployment == 'PBACO':
    shutil.move(fr"{path}\Patient Alerts by NPI and Payor.xlsx", fr"{path}\Raw_Files\Patient Alerts by NPI and Payor.xlsx")
else:
    shutil.move(fr"{path}\{deployment} Patient Alerts by NPI and Payor.xlsx", fr"{path}\Raw_Files\Patient Alerts by NPI and Payor.xlsx")
    
print(f'Step 2 Finished, file saved at: {path}\\{filename}_{date_input}.xlsx')
continue_input=input('Enter Y when you are ready to continue')
if continue_input.lower=='y':
    #print('Start looping consultant') #currently payor doesn't need to be loop for each pc
    sys.exit() 
else:
    sys.exit()
    

Step 2 Finished, file saved at: C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\1223-1229 Empire\Empire_Alerts_by_Payor_1223-1229.xlsx


Enter Y when you are ready to continue Y


SystemExit: 

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3468: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# **--DO NOT USE Part 3 Loop Consultant**

In [ ]:
# #unique_consultants = sorted(df['Provider Consultant'].unique())
# all_consultants = pd.read_excel(r"C:\Users\ZemingLiu\OneDrive - PBACO\Documents\Weekly App Reports\Derrell Macros\Patient Alerts Macro\Derrell Patient Alerts Macro.xlsm",sheet_name='Consultants')
# unique_consultants = sorted(all_consultants[all_consultants['ACO']==deployment]['Consultant'].tolist())

# for consultant in unique_consultants:
#     # Copy the original worksheet
#     new_sheet = body_wb.copy_worksheet(body_ws)
#     new_sheet.title = consultant
    
#     # Iterate over rows starting from row 7 
#     rows_to_delete = []
#     for idx, row in enumerate(new_sheet.iter_rows(min_row=7, max_row=new_sheet.max_row, values_only=True), start=7):
#         if row[5] != consultant: # Check if the value in column F (index 5) matches the consultant name
#             rows_to_delete.append(idx)  # Use idx (row number) instead of row[0]

#     # Remove rows from bottom to top to avoid index shifting
#     for row_num in sorted(rows_to_delete, reverse=True):
#         new_sheet.delete_rows(row_num)
#         new_sheet['A4'] = None

In [ ]:
# #export
# body_wb.save(fr'{path}\{filename}_{date_input}.xlsx')
# print(f'Step 3 Loop Consultant Finished, file saved at: {path}\{filename}_{date_input}.xlsx')
# header_wb.close()
# body_wb.close()

In [ ]:
# continue_input=input('Enter Y when you are ready to continue')
# if  continue_input == 'Y' or continue_input == 'y':
#     print('Start Step 4,Save Excel and PDF')
# else:
#     sys.exit()

# **--DO NOT USE Part 4 Save Excel And PDF**

In [ ]:
# import os
# import win32com.client as win32

In [ ]:
# # open workbook
# excel_app = win32.Dispatch('Excel.Application')
# excel_app.Visible = False  # Make the application invisible
# workbook = excel_app.Workbooks.Open(fr'{path}\{filename}_{date_input}.xlsx')

In [ ]:
# #save the copy and print for each sheet
# if deployment=='PBACO':
#     folder_path=fr'{path}\patient_alerts_{date_input}'
# else:
#     folder_path=fr'{path}\patient_alerts_{deployment}_{date_input}'
# os.makedirs(folder_path, exist_ok=True)

# for sheet in workbook.Sheets:
#     sheetname=sheet.Name
    
#     #save copy for each sheet
#     sheet.Copy()
#     new_workbook = excel_app.ActiveWorkbook
#     new_workbook.SaveAs(fr'{folder_path}\patient_alerts_{sheetname}_{date_input}.xlsx')
    
#     # Print to PDF
#     sheet.PageSetup.Orientation = 2  # Landscape
#     sheet.PageSetup.PaperSize = 5  # Tabloid 11" x 17"
#     sheet.PageSetup.Zoom = False
#     sheet.PageSetup.FitToPagesWide = 1  # Fit all columns on one page
#     sheet.PageSetup.FitToPagesTall = False  # Let rows span multiple pages
#     sheet.PageSetup.LeftMargin = 0.75
#     sheet.PageSetup.RightMargin = 0.75
#     sheet.PageSetup.TopMargin = 0.75
#     sheet.PageSetup.BottomMargin = 0.75
#     if deployment=='PBACO':
#         sheet.ExportAsFixedFormat(0, fr'{folder_path}\patient_alerts_{sheetname}_{date_input}.pdf')
#     else:
#         sheet.ExportAsFixedFormat(0, fr'{folder_path}\patient_alerts_{sheetname}_{deployment}_{date_input}.pdf')
    
#     # Close the new workbook
#     new_workbook.Close(SaveChanges=False)

In [ ]:
# # Close the workbook
# print(f'Step 4 Save Excel And PDF finished, file saved at: {folder_path}, move app subscription, dashboard and low_app_performers_ongoing files to destination folder for the S3')
# workbook.Close(SaveChanges=False)
# excel_app.Quit()